<a href="https://colab.research.google.com/github/rafajak/gpt3_examples/blob/master/content_filter_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install openai
!pip install transformers

from google.colab import files
import openai
from transformers import GPT2Tokenizer
import pandas as pd
import numpy as np

In [88]:
# To keep your secret API key secure - create a api_key.json file containing the key and upload it with the function below. 
# My config file looks like this: {"api_key": "heresmysecretkey"}

upload = files.upload()
openai.api_key = json.load(open("api_key.json", "r"))["api_key"]

Saving api_key.json to api_key (2).json


In [114]:
#arguments to send the API
text_to_classify = "Dang, this falafel is freaking fantastic"

kwargs = {"engine":"content-filter-alpha-c4",
          "temperature":0,
          "max_tokens":1,
          "top_p":0,
          "logprobs":3,
          "prompt":f"<|endoftext|>{text_to_classify}\n--\nLabel:"
          }

r = openai.Completion.create(**kwargs)


    0 - The text is safe.

    1 - This text is sensitive. This means that the text could be talking 
        about a sensitive topic, something political, religious, or talking
        about a protected class such as race or nationality.
        
    2 - This text is unsafe. This means that the text contains profane
        language, prejudiced or hateful language, something that could be 
        NSFW, or text that portrays certain groups/people in a harmful manner.



In [115]:
#Let's see how did the content classifier label the text (text_to_classify)
outputs = pd.DataFrame(r["choices"][0]["logprobs"]["top_logprobs"]).T
outputs.index = ["safe_0","sensitive_1","unsafe_2"]
outputs

,0
safe_0,-0.424900
sensitive_1,-3.065525
unsafe_2,-1.206150


In [116]:
print(f"Text '{text_to_classify}' was classified as {outputs.idxmax()[0]}")

Text 'Dang, this falafel is freaking fantastic' was classified as safe_0


In [117]:
#View as probabilities:
outputs = outputs.T
outputs = outputs.apply(lambda x: np.round(np.e**x,2))
outputs

,safe_0,sensitive_1,unsafe_2
0,0.65,0.05,0.3
